# Gemini API: GitHub Issues Viewer

This notebook demonstrates how to use the Gemini API with function calling to interact with the GitHub API and display information about issues in a repository.

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/GitHub_Issues_Viewer.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

## Setup

### Install dependencies

First, install the necessary Python libraries. `google-genai` is for the Gemini API, and `PyGithub` is for interacting with the GitHub API.

In [ ]:
%pip install -qU 'google-genai' 'PyGithub'

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

You will also need a GitHub Personal Access Token stored as a Colab Secret named `GITHUB_API_KEY`. See the [GitHub documentation](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token) for instructions on how to create a token.

In [ ]:
from google import genai
import os

try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    GITHUB_API_KEY = userdata.get("GITHUB_API_KEY")
except (ModuleNotFoundError, ImportError):
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    GITHUB_API_KEY = os.environ.get("GITHUB_API_KEY")

if not GOOGLE_API_KEY or not GITHUB_API_KEY:
    raise ValueError("Please set the GOOGLE_API_KEY and GITHUB_API_KEY as environment variables or Colab secrets.")

client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

Next, choose a Gemini model that supports function calling. In this example, we'll use `gemini-2.5-flash-preview-05-20`.

In [ ]:
MODEL_ID="gemini-2.5-flash-preview-05-20"

## Define GitHub API functions as tools

Now, define the Python functions that will be used as tools by the Gemini model. These functions will interact with the GitHub API using the `PyGithub` library.

The `get_github_issues` function takes a repository name as input and returns a list of open issues. The docstring of the function is important, as it provides the model with the necessary information to understand what the function does and what arguments it expects.

In [ ]:
from github import Github

g = Github(GITHUB_API_KEY)

def get_github_issues(repository_name: str):
    """Gets the open issues for a given GitHub repository.

    Args:
        repository_name: The name of the repository in the format 'owner/repo'.
    """
    repo = g.get_repo(repository_name)
    issues = repo.get_issues(state='open')
    issue_list = []
    for issue in issues:
        issue_list.append({
            'title': issue.title,
            'number': issue.number,
            'url': issue.html_url
        })
    return issue_list

## Interact with the model

Now it's time to interact with the Gemini model. Create a `ChatSession` and pass the `get_github_issues` function to the `tools` parameter. Then, send a prompt to the model asking it to retrieve the open issues from a specific repository.

The model will automatically call the `get_github_issues` function with the correct repository name, and then use the returned information to generate a user-friendly response.

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": [get_github_issues]
    }
)

response = chat.send_message("Show me the open issues in the google/gemini-api-cookbook repository.")

print(response.text)